In [1]:
import psutil

def get_available_memory():
    # Get the available virtual memory in bytes
    virtual_memory = psutil.virtual_memory()
    
    # Convert the available memory from bytes to human-readable format (e.g., MB or GB)
    available_memory = virtual_memory.available / (1024 ** 3)  # Convert to GB
    
    return available_memory

if __name__ == "__main__":
    available_memory = get_available_memory()
    print(f"Available Memory: {available_memory:.2f} GB")


Available Memory: 4.56 GB


In [ ]:

import torch

'''
def main(local_rank):
    torch.cuda.set_device(local_rank)
    device = torch.device('cuda', local_rank)

if __name__ == "__main__":
    print (torch.__version__)
    print (torch.cuda.is_available())
    #main(0)
'''

In [ ]:
print("hello world")
s = set()
for i in range(100000000):
    s.add(i)
    if i % 1000000 == 0:
        print(i)

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

/Users/allenbaranov/opt/anaconda3/envs/news/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [8]:
from transformers import pipeline
import numpy as np

class SentimentAnalysis():
    def __init__(self):
        self.model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

    def get_sentiment_score(self, text):
        out = self.model(text)[0]
        return out['label']

In [9]:
model = SentimentAnalysis()

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [16]:
for text in ['i hate icecream', 'neutral', 'i love puppies', 'Israel-Gaza live updates: 1 dead, 17 injured in car-ramming attacks near Tel Aviv']:
    print(model.get_sentiment_score(text))

NEG
NEU
POS
NEG


In [25]:
classifier =  pipeline("zero-shot-classification", model='roberta-base')
def classify_text_with_confidence(text, candidate_labels):
    # Perform zero-shot classification
    result = classifier(text, candidate_labels)
    return dict(zip(result['labels'], map(lambda x: f"{x:.0%}", result['scores'])))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [30]:
text = "The super PAC supporting Gov. Ron DeSantis of USA" #Florida in the presidential race began carrying out layoffs on Wednesday, even as the campaign insisted he had a path forward. It was unclear how broad the layoffs were at the super PAC, Never Back Down. The group had spent heavily on a vast field operation in Iowa, taking over many of the responsibilities of a traditional campaign, but Mr. DeSantis lost the state’s caucuses to former President Donald J. Trump on Monday by 30 percentage points. One of those who was laid off, George Andrews, who had been assigned as a caucus precinct operations director in Iowa but also listed himself on LinkedIn as a state director in California, posted on the career website that he had been let go. “As of 6 am this morning, I learned I am now a free agent due to budget cuts beyond my control,” Mr. Andrews wrote in a post on LinkedIn. “I completely understand why this had to happen, harbor no ill will, and wish my former team great success as they attempt to bring back sanity to our party,” he wrote. “What they are trying to accomplish for America is much greater than my termination as an individual employee.” An official with the group appeared to confirm the layoffs, saying that those affected were being paid through the end of January. The official, who did not speak on the record, added that the group was “evaluating and paring down” other consultants, vendors and some staff members who had been focused on various aspects of the group’s work. Scott Wagner, the chief executive of Never Back Down, issued a statement saying that the group continued to host events for Mr. DeSantis, but he did not address the question of layoffs. “Never Back Down continues to host a slew of events on the ground for Gov. DeSantis in South Carolina, New Hampshire, and beyond aligned with our core mission of mobilizing grass-roots field operations in those state,” Mr. Wagner said. “We’ve mobilized several members of our robust Iowa team over to the other early primary states to help in these efforts and will continue working to help elect Gov. DeSantis, the most effective conservative leader in the race, our next President.” "
#labels = ["informative", "formal", "objective", "serious", "persuasive", "informal", "opinionated", "satirical"]

def classify_text(text):
    # Perform zero-shot classification
    result = classifier(text)
    return dict(zip(result['labels'], map(lambda x: f"{x:.0%}", result['scores'])))
confidences = classify_text(text)#, labels)
print("Confidence Scores:")
for label, confidence in confidences.items():
    print(f"{label}: {confidence}")

TypeError: object of type 'NoneType' has no len()

In [35]:
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
labels = ["optimism, pessimism"]


model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Good night 😊"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


1) joy 0.9061
2) optimism 0.0407
3) sadness 0.0406
4) anger 0.0126


In [38]:
text = "The super PAC supporting Gov. Ron DeSantis of Florida in the presidential race began carrying out layoffs on Wednesday, even as the campaign insisted he had a path forward. It was unclear how broad the layoffs were at the super PAC, Never Back Down. The group had spent heavily on a vast field operation in Iowa, taking over many of the responsibilities of a traditional campaign, but Mr. DeSantis lost the state’s caucuses to former President Donald J. Trump on Monday by 30 percentage points. One of those who was laid off, George Andrews, who had been assigned as a caucus precinct operations director in Iowa but also listed himself on LinkedIn as a state director in California, posted on the career website that he had been let go. “As of 6 am this morning, I learned I am now a free agent due to budget cuts beyond my control,” Mr. Andrews wrote in a post on LinkedIn. “I completely understand why this had to happen, harbor no ill will, and wish my former team great success as they attempt to bring back sanity to our party,” he wrote. “What they are trying to accomplish for America is much greater than my termination as an individual employee.” An official with the group appeared to confirm the layoffs, saying that those affected were being paid through the end of January. The official, who did not speak on the record, added that the group was “evaluating and paring down” other consultants, vendors and some staff members who had been focused on various aspects of the group’s work. Scott Wagner, the chief executive of Never Back Down, issued a statement saying that the group continued to host events for Mr. DeSantis, but he did not address the question of layoffs. “Never Back Down continues to host a slew of events on the ground for Gov. DeSantis in South Carolina, New Hampshire, and beyond aligned with our core mission of mobilizing grass-roots field operations in those state,” Mr. Wagner said. “We’ve mobilized several members of our robust Iowa team over to the other early primary states to help in these efforts and will continue working to help elect Gov. DeSantis, the most effective conservative leader in the race, our next President.” "

encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


IndexError: list index out of range

In [28]:
import openai
import numpy as np
from transformers import pipeline
from textblob import TextBlob
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

from dotenv import load_dotenv
import os
load_dotenv("../")


openai_api = os.environ.get('OPENAI_API')



class SemanticEmbeddings:
    def __init__(self):
        self.engine = 'text-embedding-ada-002'
        self.api = openai_api

    def get_embedding(self, text, engine = 'text-embedding-ada-002'):
        """
        Get the embedding for the given text using OpenAI's Embedding API.

        :param text: The text to embed.
        :param engine: The embedding engine to use.
        :return: Embedding vector.
        """
        openai.api_key = self.api

        response = openai.Embedding.create(
            input=text,
            engine=engine
        )

        embedding = response["data"][0]["embedding"]
        return embedding

    def similarity_score(self, x, y, verbose = True):
        x = np.array(x)
        y = np.array(y)
        sim_score = x.T@y / (np.linalg.norm(x) * np.linalg.norm(y))
        if verbose:
            print(f"Similarity between the two embeddings is: {sim_score:.4f}")
        return sim_score
        


class SentimentAnalysis():
    def __init__(self):


        # Tasks:
        # emoji, emotion, hate, irony, offensive, sentiment
        # stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

        task='sentiment'
        MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

        self.tokenizer = AutoTokenizer.from_pretrained(MODEL)

        # download label mapping
        labels=[]
        mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
        with urllib.request.urlopen(mapping_link) as f:
            html = f.read().decode('utf-8').split("\n")
            csvreader = csv.reader(html, delimiter='\t')
        self.labels = [row[1] for row in csvreader if len(row) > 1]

        self.model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
        self.model.save_pretrained(MODEL)

    
    def get_sentiment_score(self, text):

        #text = "Good night 😊"
        encoded_input = self.tokenizer(text, return_tensors='tf')
        output = self.model(encoded_input)
        scores = output[0][0].numpy()
        scores = softmax(scores)

        out = 0
        for ranking in scores:
            if ranking['label'] == 'positive':
                out += ranking['score']
            if ranking['label'] == 'negative':
                out -= ranking['score']

        return out
        '''
        out = self.model(text)[0]
        s = out['score']
        b = 2*s**3
        c = np.sqrt(1+b**2)
        a = -1/np.log(c-b)
        
        x = TextBlob(text).sentiment.polarity # blob sentiment
        
        if out['label'] == 'NEU':
            b = 0
        elif out['label'] == 'NEG':
            b = -b
        
        return a*np.log(b*x+c)
        '''

class ToneAnalysis():
    def __init__(self):
        pass

    def categorize(self, text):
        categories = ['informative','objective','serious','']

if __name__ == '__main__':
    a = SentimentAnalysis()
    b = SemanticEmbeddings()
    c = ToneAnalysis()


/Users/allenbaranov/opt/anaconda3/envs/news/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-19 19:22:36.669632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'textblob'

In [21]:
s='''["Trump's Warning to Supreme Court Regarding Colorado Ballot", "Lowest Home Sales in US in 28 Years", "Winter Storm Impacting NYC Area", "Jetblue Cutting Routes for Profitability ", "Japan's Successful Lunar Landing", "Donald Trump's Endorsement from Tim Scott", "Investigation into Atlas Airline Incident in Miami", "Rise of Young Girls Frequents Sephora Stores", "Effects of Israel’s War on Gaza", "Changes in New Hampshire Primary Landscape", "Job Losses at Sports Illustrated", "Buffalo Bills' Call for Snow Shoveling Assistance"]'''
q=s[1:-1].replace(' "','').replace('"','').split(",")

In [22]:
q


["Trump's Warning to Supreme Court Regarding Colorado Ballot",
 'Lowest Home Sales in US in 28 Years',
 'Winter Storm Impacting NYC Area',
 'Jetblue Cutting Routes for Profitability',
 "Japan's Successful Lunar Landing",
 "Donald Trump's Endorsement from Tim Scott",
 'Investigation into Atlas Airline Incident in Miami',
 'Rise of Young Girls Frequents Sephora Stores',
 'Effects of Israel’s War on Gaza',
 'Changes in New Hampshire Primary Landscape',
 'Job Losses at Sports Illustrated',
 "Buffalo Bills' Call for Snow Shoveling Assistance"]

In [20]:
q = [str(x) for x in q]
print(q)

["Trump's Warning to Supreme Court Regarding Colorado Ballot", 'Lowest Home Sales in US in 28 Years', 'Winter Storm Impacting NYC Area', 'Jetblue Cutting Routes for Profitability', "Japan's Successful Lunar Landing", "Donald Trump's Endorsement from Tim Scott", 'Investigation into Atlas Airline Incident in Miami', 'Rise of Young Girls Frequents Sephora Stores', 'Effects of Israel’s War on Gaza', 'Changes in New Hampshire Primary Landscape', 'Job Losses at Sports Illustrated', "Buffalo Bills' Call for Snow Shoveling Assistance"]


In [9]:
from dotenv import load_dotenv
import os

load_dotenv('../.env')  # This loads the variables from .env

print(os.environ.get('BING_API'))

None


In [11]:
import os
cwd = os.getcwd()
print("Current working directory:", cwd)


Current working directory: /Users/allenbaranov/Desktop/2024 IAP UROP/next-week-tonight/Scraps


In [21]:
relative_dotenv_path = os.path.join(cwd, '../vars.env')

In [22]:
# Resolve to the absolute path
dotenv_path = os.path.abspath(relative_dotenv_path)
print("Resolved .env path:", dotenv_path)


Resolved .env path: /Users/allenbaranov/Desktop/2024 IAP UROP/next-week-tonight/vars.env


In [26]:
from dotenv import load_dotenv

print(dotenv_path)
load_dotenv("../")

print(os.environ.get('BING_API'))

/Users/allenbaranov/Desktop/2024 IAP UROP/next-week-tonight/vars.env
v5cde7c3a668e4cb2bd42e622495d230b


/Users/allenbaranov/Desktop/2024 IAP UROP/next-week-tonight/vars.env
v5cde7c3a668e4cb2bd42e622495d230b
